Test net

In [1]:
store_list=['MD563','JOG03','MD387']
placeholders =  ", ".join([f"'{s}'" for s in store_list])
# df1['kode_list'] = df1['kode'].str.split(',')

# 2. Explode untuk meratakan
# df1_exploded = df1.explode('kode_list')

In [5]:
from django.db import connection
def execute_raw_sql(query, params=None):
    with connection.cursor() as cursor:
        cursor.execute(query, params or [])
        columns = [col[0] for col in cursor.description]  # Get column names from cursor description
        result = cursor.fetchall()

        # Convert result to a list of dictionaries where column names are keys
        formatted_result = [dict(zip(columns, row)) for row in result]
    return formatted_result
dbLoad = 'default'
dbsave = 'local'

In [8]:
def df__customers_pb_n(placeholders):
        query = f'''
            SELECT fn_cusid,fn_whconsiid FROM gis_local.customers_pb
            where fc_code in ({placeholders})
            '''
        results = execute_raw_sql(query)
        df = pd.DataFrame(results)
        return df
df_cus=df__customers_pb_n(placeholders)
cus_list= df_cus['fn_cusid'].tolist()
cus_list =  ", ".join([f"'{s}'" for s in cus_list])
whconsi_list=df_cus['fn_whconsiid'].tolist()
whconsi_list =  ", ".join([f"'{s}'" for s in whconsi_list])
def change_cusid_to_whconsiid(df):
    df_cus=df__customers_pb_n(placeholders)
    df=pd.merge(df,df_cus,how='left',on='fn_cusid')
    df['fn_cusid']=df['fn_whconsiid']
    df=df.drop(columns='fn_whconsiid')
    df=df.rename(columns={'fn_cusid':'fn_whconsiid'})
    return df
def get_ostdo_(cus_list):
    query = f'''
        SELECT fv_artsizecode,fv_barcode from artikel_pb
        '''
    results = execute_raw_sql(query)
    df_art = pd.DataFrame(results)
    query_1= f'''SELECT 
    c.fn_cusid ,fv_artsizecode, sum(fn_qty) fn_qty
    FROM gis_db.dodtl_tb a 
    inner join gis_db.domst_tb b on a.fc_nodoc=b.fc_nodoc
    inner join gis_db.t_ordermst c on b.fc_noreff=fv_noorder
    left join gis_db.t_receivingmst d on b.fc_nodoc=d.fc_nosj
    where  fv_noreceiving is null and c.fn_cusid in ({cus_list}) and b.fc_status="F" 
    group by fn_cusid, fv_artsizecode, fd_date 
    '''
    # DatabaseConnector.execute_raw_sql(OutstandingDO_query, using=StockDashboardAPIView.dbsave)
    results_1 = DatabaseConnector.execute_raw_sql(query_1, using=StockDashboardAPIView.dbsave)
    df = pd.DataFrame(results_1)
    df=change_cusid_to_whconsiid(df)
    df=pd.merge(df, df_art, on='fv_artsizecode', how='left')
    df=df.groupby(['fn_whconsiid','fv_barcode',], as_index=False)['fn_qty'].sum()
    return df
OutstandingDO_df=get_ostdo_(cus_list)

ImproperlyConfigured: Requested setting DATABASES, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.